# Launch LBANN traing jobs
Code to create a batch script for launching jobs on cori GPU

Sep 1, 2020

In [1]:
import os
import glob,time
import subprocess as sp
import numpy as np

import yaml


In [29]:
start_dir=os.getcwd()
print(start_dir)

/autofs/nccs-svm1_home1/venkitesh/projects/cosmogan/cosmogan_pytorch/code/run_scripts


## Define machine and code 

In [111]:
facility='summit'
img_size=128
run_type='2dgan'
assert facility in ['cori','summit'], "Error%s"%(facility)
assert img_size in [64,128,512]
assert run_type in ['2dgan','2dcgan','3dgan','3dcgan']

## Read ref_launch.yaml and define dictionary

In [112]:
## Read ref_launch.yaml
# start_dir='/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/run_scripts'
launch_cfile=start_dir+'/ref_launch.yaml'

with open(launch_cfile) as f:
    config_dict= yaml.load(f, Loader=yaml.SafeLoader)

main_code_dir=config_dict[facility]['main_code_dir']
dict_pars=config_dict[facility][img_size][run_type]
print(dict_pars)
dict_pars.update({'nodes':2,'gpus_per_node':8,'time':'4:00:00','job_name':'ddp_3dgan'})

dict_pars['code_dir']=main_code_dir+dict_pars['code_dir']
dict_pars['staging_loc']=config_dict[facility]['staging_loc']
print(dict_pars)
staging_loc=dict_pars['staging_loc']

{'code_dir': 'code/1_basic_GAN/1_main_code/DDP_new_loss', 'config': 'config_2dgan_summit.yaml', 'val_file': '/gpfs/alpine/ast153/proj-shared/venkitesh/Cosmogan/data/raw_data/2d_data/128_square/dataset_2_smoothing_200k/norm_1_train_val.npy'}
{'code_dir': '/autofs/nccs-svm1_home1/venkitesh/projects/cosmogan/cosmogan_pytorch/code/1_basic_GAN/1_main_code/DDP_new_loss', 'config': 'config_2dgan_summit.yaml', 'val_file': '/gpfs/alpine/ast153/proj-shared/venkitesh/Cosmogan/data/raw_data/2d_data/128_square/dataset_2_smoothing_200k/norm_1_train_val.npy', 'nodes': 2, 'gpus_per_node': 8, 'time': '4:00:00', 'job_name': 'ddp_3dgan', 'staging_loc': '/gpfs/alpine/ast153/proj-shared/venkitesh/Cosmogan/data/results_pytorch/'}


In [113]:
cori_strg='''#!/bin/bash
#################
#SBATCH --nodes={nodes}
#SBATCH --qos=regular
#SBATCH --output=slurm-%x-%j.out
#SBATCH --constraint=gpu
#SBATCH --account=m3363
#SBATCH --ntasks-per-node={gpus_per_node}
#SBATCH --cpus-per-task=10
#SBATCH --gpus-per-task=1
#SBATCH --time={time}
#SBATCH --job-name={job_name}

echo "--start date" `date` `date +%s`
echo '--hostname ' $HOSTNAME

#nGPU_per_node=$SLURM_GPUS_PER_TASK 

### Initial setup
module load cgpu
module load pytorch/v1.6.0-gpu
#conda activate v3

N=${{SLURM_NNODES}}
nprocspn=${{SLURM_NTASKS_PER_NODE}}
G=$[ $N * $nprocspn ]
echo S:`hostname` G=$G  N=$N
export MASTER_ADDR=`hostname --ip-address`
echo S:use MASTER_ADDR=$MASTER_ADDR

### Run the main code
code_dir={code_dir}

export CMD=" python -u $code_dir/main.py --config $code_dir/{config} --facility cori --ddp"
srun -n $G bash -c ' ${{CMD}} '

echo "--end date" `date` `date +%s`
'''

summit_strg='''#!/bin/bash
# Begin LSF directives
#BSUB -P AST153
#BSUB -J {job_name} 
#BSUB -o lsf-{job_name}.%J
#BSUB -W {time}
#BSUB -nnodes {nodes}
#BSUB -alloc_flags "nvme smt4"
# End LSF directives and begin shell commands

NODES=$(cat ${{LSB_DJOB_HOSTFILE}} | sort | uniq | grep -v login | grep -v batch | wc -l)

module load open-ce
# add this 2 lines to run inside conda with my privately compiled Apex
module load gcc/8
conda activate cloned_env

export OMP_NUM_THREADS=1
export NCCL_DEBUG=INFO

config=bs1024-smt
ver=S  # train.py version
jobId=${{LSB_JOBID}}

code_dir={code_dir}
config_file=$code_dir/{config}

CMD=" python -u $code_dir/main.py --config $config_file --facility summit --ddp "
echo "S:CMD=$CMD"

SMPIARGS="--smpiargs "off""
jsrun -n${{NODES}} -a6 -c42 -g6 -r1 $SMPIARGS --bind=proportional-packed:7 --launch_distribution=packed ./launchSummit.sh "$CMD"
'''

In [114]:
# Build bash string
if facility=='cori':
    bash_strg=cori_strg.format(**dict_pars)
elif facility=='summit':
    bash_strg=summit_strg.format(**dict_pars)


In [115]:
fname='batch_train_{0}.sh'.format(run_type)
filename=dict_pars['staging_loc']+fname
with open (filename,'w') as f:
    f.write(bash_strg)
print(filename)

/gpfs/alpine/ast153/proj-shared/venkitesh/Cosmogan/data/results_pytorch/batch_train_2dgan.sh


In [116]:
### Move to staging locations in project space:
# os.chdir(dict_pars['staging_loc'])

In [117]:
%%bash -s "$filename" ## Use python variable in bash
cat $1
chmod +x $1

#!/bin/bash
# Begin LSF directives
#BSUB -P AST153
#BSUB -J ddp_3dgan 
#BSUB -o lsf-ddp_3dgan.%J
#BSUB -W 4:00:00
#BSUB -nnodes 2
#BSUB -alloc_flags "nvme smt4"
# End LSF directives and begin shell commands

NODES=$(cat ${LSB_DJOB_HOSTFILE} | sort | uniq | grep -v login | grep -v batch | wc -l)

module load open-ce
# add this 2 lines to run inside conda with my privately compiled Apex
module load gcc/8
conda activate cloned_env

export OMP_NUM_THREADS=1
export NCCL_DEBUG=INFO

config=bs1024-smt
ver=S  # train.py version
jobId=${LSB_JOBID}

code_dir=/autofs/nccs-svm1_home1/venkitesh/projects/cosmogan/cosmogan_pytorch/code/1_basic_GAN/1_main_code/DDP_new_loss
config_file=$code_dir/config_2dgan_summit.yaml

CMD=" python -u $code_dir/main.py --config $config_file --facility summit --ddp "
echo "S:CMD=$CMD"

SMPIARGS="--smpiargs "off""
jsrun -n${NODES} -a6 -c42 -g6 -r1 $SMPIARGS --bind=proportional-packed:7 --launch_distribution=packed ./launchSummit.sh "$CMD"


## Submit job to cori GPU

In [118]:
# %%bash -s "$filename" ## Use python variable in bash
# module load cgpu
# sbatch $1